# SVM with TFIDF vectors

In [3]:
!pip install pandas scikit-learn

  Using cached pandas-2.2.3-cp310-cp310-macosx_11_0_arm64.whl.metadata (89 kB)
  Using cached scikit_learn-1.6.1-cp310-cp310-macosx_12_0_arm64.whl.metadata (31 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached pandas-2.2.3-cp310-cp310-macosx_11_0_arm64.whl (11.3 MB)
Using cached scikit_learn-1.6.1-cp310-cp310-macosx_12_0_arm64.whl (11.1 MB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.4/22.4 MB 12.0 MB/s eta 0:00:0000:0100:01
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [pandas]2m6/7 [pandas]learn]


In [16]:
# imports here

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

In [9]:
df = pd.read_csv("./data/training.csv")
df.head()

,id,sentence,label
0,0,Those 2 drinks are part of the HK culture and ...,negative
1,1,I was told by the repair company that was doin...,negative
2,2,It is there to give them a good time .,neutral
3,3,Like leafing through an album of photos accomp...,negative
4,4,Johnny was a talker and liked to have fun.,positive


In [12]:
sentences = df['sentence'].astype(str)
labels = df['label']

In [13]:
labelEncoder = LabelEncoder()
labels = labelEncoder.fit_transform(labels)
labels

array([0, 0, 1, ..., 1, 1, 1], shape=(102097,))

In [18]:
X_train, X_test, y_train, y_test = train_test_split(sentences, labels, test_size=0.2,random_state=42)

In [19]:
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [21]:
sentiment_cls = SVC(kernel='linear')
sentiment_cls.fit(X_train_vec, y_train)

SVC(kernel='linear')

In [22]:
y_pred = sentiment_cls.predict(X_test_vec)
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.6765426052889324


In [23]:
df_test = pd.read_csv('./data/test.csv')
df_test.head()

,id,sentence
0,0,Found Thai Spoon on the Vegan Pittsburgh website.
1,1,Our bill came out to around $27 and we ate lik...
2,2,State Farm broke down the costs for me of the ...
3,3,The only con for this resto is the wait to get...
4,4,We could hear the people above us stomping aro...


In [24]:
final_test_sentences = df_test['sentence']
final_test_vect = vectorizer.transform(final_test_sentences)

In [25]:
final_y_pred = sentiment_cls.predict(final_test_vect)

In [29]:
final_submission = labelEncoder.inverse_transform(final_y_pred)

In [30]:
sub_df = pd.DataFrame({
    'id': range(len(final_submission)),
    'label': final_submission
})

sub_df.head()

,id,label
0,0,neutral
1,1,neutral
2,2,neutral
3,3,negative
4,4,neutral


In [31]:
sub_df.to_csv('submission_svm.csv', index=False)

# Pretrained Distilbert finetuned on SST2

In [33]:
!pip install datasets

  Using cached datasets-3.6.0-py3-none-any.whl.metadata (19 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached xxhash-3.5.0-cp310-cp310-macosx_11_0_arm64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py310-none-any.whl.metadata (7.2 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached async_timeout-5.0.1-py3-none-any.whl.metadata (5.1 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached multidict-6.4.3-cp310-cp310-macosx_11_0_arm64.whl.metadata (5.3 kB)
  Using cached propcache-0.3.1-cp310-cp310-macosx_11_0_arm64.whl.metadata (10 kB)
Using cached dill-0.3.8-py3-none-any.whl (116 kB)
Using cached multiprocess-0.70.16-py310-none-any.whl (134 kB)
Using cached async_timeout-5.0.1-py3-none-any.whl (6.2 kB)
Using cached multidict-6.4.3-cp310-cp310-macosx_11_0_arm64.whl (37 kB)
Using cached aiohappyeyeballs-2.6.1-py3-no

In [37]:
from datasets import load_dataset
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset
from tqdm.auto import tqdm

In [44]:
pipe = pipeline("text-classification")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use mps:0


In [45]:
# Now that you have the pipeline for sentiment analysis ready, let us perform inference on the test set that we have
import csv

def write_submission_pipeline(dataset):

    # Open file in write mode initially to write header
    with open('submission.csv', mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['id', 'label'])  # adjust column names as needed

    inp_sentences = [inp['sentence'] for inp in dataset['test']]
    inp_ids = [inp['id'] for inp in dataset['test']]
    outputs = pipe(inp_sentences) # perform inference
    print(len(inp_sentences))
    print(len(inp_ids))
    print(len(outputs))
    
    # Write output to submission
    for idx in range(len(outputs)):
        with open('submission_bert_sst2.csv', mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([inp_ids[idx], outputs[idx]['label'].lower()])


In [46]:
dataset = load_dataset("csv", data_files={"train": "./data/training.csv", "test": "./data/test.csv"})
write_submission_pipeline(dataset)

11951
11951
11951


# Logistic Regression with all-MiniLM-L6-v2 Model Embeddings

In [48]:
!pip install sentence_transformers

  Using cached sentence_transformers-4.1.0-py3-none-any.whl.metadata (13 kB)
  Using cached pillow-11.2.1-cp310-cp310-macosx_11_0_arm64.whl.metadata (8.9 kB)
Using cached sentence_transformers-4.1.0-py3-none-any.whl (345 kB)
Using cached pillow-11.2.1-cp310-cp310-macosx_11_0_arm64.whl (3.0 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [sentence_transformers]ence_transformers]


In [49]:
from sentence_transformers import SentenceTransformer

In [50]:
model_name = 'all-MiniLM-L6-v2'
print(f'Loading model: {model_name}')

model_embd = SentenceTransformer(model_name)

train_input_sentences = [s['sentence'] for s in dataset['train']]

inp_sentence_embeddings = model_embd.encode(train_input_sentences)

Loading model: all-MiniLM-L6-v2


In [ ]:
from sklearn.linear_model import LogisticRegression

# Logistic Regression Classifier, C: Inverse of regularization strength , max_iter: Maximum number of training iterations
label_map = {'negative': -1, "neutral": 0, "positive": 1}
train_encoded_labels = [label_map[s['label']] for s in dataset['train']]
model = LogisticRegression(C=1.0, max_iter=100)
model.fit(inp_sentence_embeddings, train_encoded_labels)

In [ ]:
y_train_pred = model.predict(inp_sentence_embeddings)

out_sentences = [s['sentence'] for s in dataset['test']]
out_sentences_embeddings = model_embd.encode(out_sentences)

y_test_pred = model.predict(out_sentences_embeddings)

In [ ]:
reverse_label_map = {-1: "negative", 0: "neutral", 1: "positive"}
y_test_pred = [reverse_label_map[pred] for pred in y_test_pred]

In [ ]:
from sklearn.metrics import mean_absolute_error
# Score on Training Set
mae_train = mean_absolute_error(train_encoded_labels, y_train_pred)
L_score_train = 0.5 * (2 - mae_train)
print(L_score_train)

In [ ]:
import csv

# Open file in write mode initially to write header
with open('submission_logreg_embd.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['id', 'label'])  # adjust column names as needed

# Write output to submission
for idx in range(len(outputs)):
    with open('submission_logreg_embd.csv', mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([inp_ids[idx], y_test_pred[idx]])